In [1]:
import functools

import jax
import numpy as np
import scanpy as sc

from cfp.metrics import compute_mean_metrics, compute_metrics, compute_metrics_fast
import cfp.preprocessing as cfpp

In [2]:
split = 1

In [3]:
adata_pred_ood = sc.read(f"/lustre/groups/ml01/workspace/ot_perturbation/models/otfm/combosciplex/adata_ood_with_predictions_{split}_attention_seed.h5ad")
adata_pred_test = sc.read(f"/lustre/groups/ml01/workspace/ot_perturbation/models/otfm/combosciplex/adata_test_with_predictions_{split}_attention_seed.h5ad")

In [4]:
adata_train_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_train_{split}.h5ad"
adata_test_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_test_{split}.h5ad"
adata_ood_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_ood_{split}.h5ad"

In [5]:
adata_train = sc.read(adata_train_path)
adata_test = sc.read(adata_test_path)
adata_ood = sc.read(adata_ood_path)

In [6]:
adata_ref_test = adata_test[adata_test.obs["condition"]!="control"].copy()
cfpp.centered_pca(adata_ref_test, n_comps=10)

adata_ref_ood = adata_ood[adata_ood.obs["condition"]!="control"].copy()
cfpp.centered_pca(adata_ref_ood, n_comps=10)

In [7]:
cfpp.project_pca(query_adata=adata_pred_test, ref_adata=adata_ref_test, layer="X_recon_pred")
cfpp.project_pca(query_adata=adata_test, ref_adata=adata_ref_test)
test_data_target_encoded = {}
test_data_target_decoded = {}
test_data_target_encoded_predicted = {}
test_data_target_decoded_predicted = {}
for cond in adata_test.obs["condition"].cat.categories:
    if cond == "control":
        continue
    test_data_target_encoded[cond] = adata_test[adata_test.obs["condition"] == cond].obsm["X_pca"]
    test_data_target_decoded[cond] = adata_test[adata_test.obs["condition"] == cond].X.toarray()
    test_data_target_decoded_predicted[cond] = adata_pred_test[adata_pred_test.obs["condition"] == cond].layers["X_recon_pred"]
    test_data_target_encoded_predicted[cond] = adata_pred_test[adata_pred_test.obs["condition"] == cond].obsm["X_pca"]

cfpp.project_pca(query_adata=adata_pred_ood, ref_adata=adata_ref_ood, layer="X_recon_pred")
cfpp.project_pca(query_adata=adata_ood, ref_adata=adata_ref_ood)
ood_data_target_encoded = {}
ood_data_target_decoded = {}
ood_data_target_encoded_predicted = {}
ood_data_target_decoded_predicted = {}
for cond in adata_ood.obs["condition"].cat.categories:
    if cond == "control":
        continue
    ood_data_target_encoded[cond] = adata_ood[adata_ood.obs["condition"] == cond].obsm["X_pca"]
    ood_data_target_decoded[cond] = adata_ood[adata_ood.obs["condition"] == cond].X.toarray()
    ood_data_target_decoded_predicted[cond] = adata_pred_ood[adata_pred_ood.obs["condition"] == cond].layers["X_recon_pred"]
    ood_data_target_encoded_predicted[cond] = adata_pred_ood[adata_pred_ood.obs["condition"] == cond].obsm["X_pca"]

In [8]:
test_deg_dict = {
    k: v
    for k, v in adata_train.uns["rank_genes_groups_cov_all"].items()
    if k in test_data_target_decoded_predicted.keys()
}
ood_deg_dict = {
    k: v
    for k, v in adata_train.uns["rank_genes_groups_cov_all"].items()
    if k in ood_data_target_decoded_predicted.keys()
}

In [9]:
def get_mask(x, y):
    return x[:, [gene in y for gene in adata_train.var_names]]


ood_deg_target_decoded_predicted = jax.tree_util.tree_map(get_mask, ood_data_target_decoded_predicted, ood_deg_dict)
ood_deg_target_decoded = jax.tree_util.tree_map(get_mask, ood_data_target_decoded, ood_deg_dict)

test_deg_target_decoded_predicted = jax.tree_util.tree_map(get_mask, test_data_target_decoded_predicted, test_deg_dict)
test_deg_target_decoded = jax.tree_util.tree_map(get_mask, test_data_target_decoded, test_deg_dict)


In [10]:
deg_ood_metrics = jax.tree_util.tree_map(compute_metrics, ood_deg_target_decoded, ood_deg_target_decoded_predicted)
deg_mean_ood_metrics = compute_mean_metrics(deg_ood_metrics, prefix="deg_ood_")

deg_test_metrics = jax.tree_util.tree_map(compute_metrics, test_deg_target_decoded, test_deg_target_decoded_predicted)
deg_mean_test_metrics = compute_mean_metrics(deg_test_metrics, prefix="deg_test_")


In [11]:
test_metrics_encoded = jax.tree_util.tree_map(
    compute_metrics, test_data_target_encoded, test_data_target_encoded_predicted
)
mean_test_metrics_encoded = compute_mean_metrics(test_metrics_encoded, prefix="encoded_test_")

test_metrics_decoded = jax.tree_util.tree_map(
    compute_metrics_fast, test_data_target_decoded, test_data_target_decoded_predicted
)
mean_test_metrics_decoded = compute_mean_metrics(test_metrics_decoded, prefix="decoded_test_")

In [12]:
ood_metrics_encoded = jax.tree_util.tree_map(
    compute_metrics, ood_data_target_encoded, ood_data_target_encoded_predicted
)
mean_ood_metrics_encoded = compute_mean_metrics(ood_metrics_encoded, prefix="encoded_ood_")

ood_metrics_decoded = jax.tree_util.tree_map(
    compute_metrics, ood_data_target_decoded, ood_data_target_decoded_predicted
)
mean_ood_metrics_decoded = compute_mean_metrics(ood_metrics_decoded, prefix="decoded_ood_")

In [13]:
mean_ood_metrics_decoded

{'decoded_ood_r_squared': 0.9700107006962718,
 'decoded_ood_sinkhorn_div_1': 117.82546670096261,
 'decoded_ood_sinkhorn_div_10': 70.47755868094308,
 'decoded_ood_sinkhorn_div_100': 3.088749476841518,
 'decoded_ood_e_distance': 5.430164846223572,
 'decoded_ood_mmd': 0.011631242598273925}

In [14]:
mean_ood_metrics_encoded

{'encoded_ood_r_squared': 0.824279584280777,
 'encoded_ood_sinkhorn_div_1': 5.067402499062674,
 'encoded_ood_sinkhorn_div_10': 2.8147490365164622,
 'encoded_ood_sinkhorn_div_100': 2.0605531420026506,
 'encoded_ood_e_distance': 3.9391318239630047,
 'encoded_ood_mmd': 0.02499505504965782}

In [15]:
deg_mean_ood_metrics

{'deg_ood_r_squared': 0.9644582615014661,
 'deg_ood_sinkhorn_div_1': 11.862500326974052,
 'deg_ood_sinkhorn_div_10': 1.528451374598912,
 'deg_ood_sinkhorn_div_100': 1.0472033364432198,
 'deg_ood_e_distance': 2.005396454979029,
 'deg_ood_mmd': 0.011631671538842576}

In [16]:
deg_mean_test_metrics

{'deg_test_r_squared': 0.9865954204624917,
 'deg_test_sinkhorn_div_1': 13.03375655412674,
 'deg_test_sinkhorn_div_10': 1.4380208651224773,
 'deg_test_sinkhorn_div_100': 0.47717122236887616,
 'deg_test_e_distance': 0.8315845834188177,
 'deg_test_mmd': 0.012474625332591435}

In [17]:
ood_metrics_encoded

{'Givinostat+Dasatinib': {'r_squared': 0.4973172955314429,
  'sinkhorn_div_1': 5.809219837188721,
  'sinkhorn_div_10': 4.23637580871582,
  'sinkhorn_div_100': 3.734792709350586,
  'e_distance': 7.33699234307657,
  'mmd': 0.03228922},
 'Givinostat+SRT1720': {'r_squared': 0.9609430328589009,
  'sinkhorn_div_1': 1.9774103164672852,
  'sinkhorn_div_10': 0.46679115295410156,
  'sinkhorn_div_100': 0.2878608703613281,
  'e_distance': 0.5490268537425749,
  'mmd': 0.005863361},
 'Panobinostat+Dasatinib': {'r_squared': 0.9303457921994389,
  'sinkhorn_div_1': 5.927687168121338,
  'sinkhorn_div_10': 2.868234634399414,
  'sinkhorn_div_100': 1.6770133972167969,
  'e_distance': 3.0714510144054543,
  'mmd': 0.027362058},
 'Panobinostat+PCI-34051': {'r_squared': 0.9032620019245348,
  'sinkhorn_div_1': 7.336724281311035,
  'sinkhorn_div_10': 4.046356201171875,
  'sinkhorn_div_100': 2.5165491104125977,
  'e_distance': 4.640937946164225,
  'mmd': 0.03397535},
 'Panobinostat+SRT2104': {'r_squared': 0.95636

In [18]:
output_dir = "/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/results/otfm"

In [19]:
import os
import pandas as pd

pd.DataFrame.from_dict(ood_metrics_encoded).to_csv(os.path.join(output_dir, f"ood_metrics_encoded_{split}_attention_seed.csv"))
pd.DataFrame.from_dict(ood_metrics_decoded).to_csv(os.path.join(output_dir, f"ood_metrics_decoded_{split}_attention_seed.csv"))
pd.DataFrame.from_dict(test_metrics_encoded).to_csv(os.path.join(output_dir, f"test_metrics_encoded_{split}_attention_seed.csv"))
pd.DataFrame.from_dict(test_metrics_decoded).to_csv(os.path.join(output_dir, f"test_metrics_decoded_{split}_attention_seed.csv"))
pd.DataFrame.from_dict(deg_test_metrics).to_csv(os.path.join(output_dir, f"test_metrics_deg_{split}_attention_seed.csv"))
pd.DataFrame.from_dict(deg_ood_metrics).to_csv(os.path.join(output_dir, f"ood_metrics_ood_{split}_attention_seed.csv"))
